In [1]:
%load_ext autoreload
%autoreload 2

## Part II: Model Building

Here you try your hand at model building to predict appointment no shows.

### Preprocessing

Package 'noshow_lib' now includes code to carry out preprocessing steps from part I. Here's how to use it:

In [2]:
import noshow_lib.util as utils

First, it includes a dictionary used for configuring path and file names
used through the project

In [3]:
utils.file_config

{'raw_data_path': 'data',
 'raw_data_csv': 'KaggleV2-May-2016.csv',
 'processed_data_path': 'processed_data',
 'train_csv': 'train_set.csv',
 'test_csv': 'test_set.csv',
 'objstore_path': 'objects',
 'feature_pipeline_file': 'feature_pipeline.pkl',
 'labels_pipeline_file': 'labels_pipeline.pkl'}

`feature_pipeline_file`: file storing the preprocessing pipeline used for preparing the feature matrix

`labels_pipeline_file`: file storing the preprocessing pipeline used for
preparing labels

`objstore_path`: directory to store python objects to disk

`processed_data_path`: directory containing processed data

`raw_data_csv`: name of the csv download from Kaggle

`raw_data_path`: directory containing raw data

`test_csv`: name of csv file containing test set

`train_csv`: name of csv file containing train set

You can change these paths and names to suit your project directory structure if you need so. E.g.,

In [4]:
file_config = utils.file_config
#config['raw_data_path'] = "some_other_directory"

First step is to create train test sets. Code is in file `noshow_lib/util.py` function `make_train_test_sets`. You
can edit that function as needed to include your own part I code if you so desire. The result will be to 
create files `train_set.csv` and `test_set.csv` in your `processed_data` directory (unless you change any of the entries in the configuration directory as above).

In [5]:
# ONLY NEED TO RUN THIS STEP ONCE (switch this to True to run it)
RUN_MAKE_TRAIN_TEST_FILES = True
if RUN_MAKE_TRAIN_TEST_FILES:
    utils.make_train_test_sets(config=file_config)

FileNotFoundError: File b'data/KaggleV2-May-2016.csv' does not exist

Next step is to fit the preprocessing pipelines. This is done in file `noshow_lib/preprocess.py`. Again you can edit code as needed in that file to incorporate your part I solution as you wish. The result will be to create files `feature_pipeline.pkl` and `labels_pipeline.pkl` containing the fit preprocessing pipelines we can then use to preprocess data.

In [6]:
import noshow_lib.preprocess as preprocess

# ONLY NEED TO RUN THIS STEP ONCE
RUN_FIT_PREPROCESSING = True
if RUN_FIT_PREPROCESSING:
    preprocess.fit_save_pipelines(config=file_config)

ModuleNotFoundError: No module named 'sklearn_pandas'

Finally, once we do that, we can get a training matrix and labels:

In [9]:
train_X, train_y = preprocess.load_train_data(config=file_config)

In [10]:
print(train_X.shape)
print(train_y.shape)

(90526, 101)
(90526,)


### Model Building

Using `sklearn` fit:
    - DecisionTree classifier
    - RandomForest classifier
    - Linear SVM classifier
    - SVM with Radial Basis Kernel classifier
    
Use default parameters for now.
Using 10-fold cross validation report both accuracy and AUC for each of the above four models.

QUESTION: Should you use accuracy or AUC for this task as a performance metric?

_ANSWER HERE_

In [7]:
# build your models here

#Decision Tree

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

DT_model = DecisionTreeClassifier()
DT_model.fit(train_X, train_y)

#Accuracy Eval
DT_model_accuracy = cross_val_score(DT_model, train_X, train_y, scoring="accuracy", cv=10)
DT_model_auc = cross_val_score(DT_model, train_X, train_y, scoring="roc_auc", cv=10)

print(DT_model_accuracy)
print(DT_model_auc)

NameError: name 'train_X' is not defined

In [ ]:
#Random Forest Classifier

from sklearn.ensemble import RandomForestClassifier


RF_model = RandomForestClassifier(n_estimators=100)
RF_model.fit(train_X, train_y)

#Accuracy Eval
RF_model_accuracy = cross_val_score(RF_model, train_X, train_y, scoring="accuracy", cv=10)
RF_model_auc = cross_val_score(RF_model, train_X, train_y, scoring="roc_auc", cv=10)

print(RF_model_accuracy)
print(RF_model_auc)



In [ ]:
# Linear SVM Classifier

from sklearn.svm import LinearSVC


LSVM_model = LinearSVC(kernel="linear")
LSVM_model.fit(train_X, train_y)

#Accuracy Eval
LSVM_model_accuracy = cross_val_score(LSVM_model, train_X, train_y, scoring="accuracy", cv=10)
LSVM_model_auc = cross_val_score(LSVM_model, train_X, train_y, scoring="roc_auc", cv=10)

print(LSVM_model_accuracy)
print(LSVM_model_auc)


In [ ]:
#RBF SVM

RBSVM_model = LinearSVC(kernel="rbf")
RBSVM_model.fit(train_X, train_y)

#Accuracy Eval
RBSVM_model_accuracy = cross_val_score(RBSVM_model, train_X, train_y, scoring="accuracy", cv=10)
RBSVM_model_auc = cross_val_score(RBSVM_model, train_X, train_y, scoring="roc_auc", cv=10)

print(RBSVM_model_accuracy)
print(RBSVM_model_auc)

### Model Tuning

Based on the above, choose two methods and fit a tuned model:
    - use 5-fold cross validation for model selection
    - use 10-fold cross validation for model assessment (based on appropriate performance metric)

Report estimated performance for both tuned classifiers

In [ ]:
# tune your models here
import pandas as pd
from sklearn.model_selection import GridSearchCV

param_grid = [
    {'loss':['hinge', 'squared_hinge'],'max_features': [5, 10, 18, 25]},
    {'bootstrap': [False], 'n_estimators': [3, 30, 60,90,100], 'max_features': [5, 10, 18, 25, 30]}
]


# Ranndom_Forest Tuning

grid_search = GridSearchCV(RF_model, param_grid=param_grid, scoring="roc_auc", cv=5)
grid_search.fit(train_X, train_y)
print ("Random_Forest: ", grid_search.best_params_)
print("AUC score is {}".format(grid_search.best_score_))

pd.DataFrame(grid_search.cv_results_)
feature_importances = grid_search.best_estimator_.feature_importances_


# Linear SVM Tuning with 10-fold
LSVM_Score= cross_val_score(LinearSVC(C=10,loss='squared_hinge',verbose=1), train_X, train_y, cv=10, scoring='roc_auc')
print(LSVM_Score)


# RBF SVM Tuning

vals = [0.0001, 0.01, 0.1, 1, 10]
param_grid = {'C': vals}
RBSVM_GS = GridSearchCV(estimator=RBSVM_model, param_grid=param_grid, scoring="roc_auc",cv=5)
SVM_GS.fit(train_X, train_y)



### Linear SVM with Gradient Descent

In [8]:
import numpy as np

# initialize model parameters w and b
# intializing to 0 is not a good idea
# it should be a random vector see np.random.randn
# YOU NEED TO IMPLEMENT THIS
def _initialize_parameters(nfeatures):
    w = np.full((nfeatures), 0.0)
    b = np.full((1), 0.0)
    return w, b

# this is a vectorized version of positive_part operation
# we can use this for hinge loss as post_part(1.0 - y*f)
pos_part = np.vectorize(lambda u: u if u > 0. else 0.)

# compute the value of the linear SVM objective function
# given current signed distances, and parameter vector w
def _get_objective(f, y, w, lam):
    loss = np.sum(pos_part(1.0 - y*f))
    penalty = lam * np.dot(w,w)
    return loss + penalty

# compute the signed distances
# based on current model estimates
# w and b
# YOU NEED TO IMPLEMENT THIS
def _get_signed_distances(X, w, b):
    nobs = X.shape[0]
    f = np.full(nobs, 0.0)
    return f

# compute gradients with respect to w and b
# YOU NEEED TO IMPLEMENT THIS
def _get_gradients(f, X, y, w, b, lam):
    nfeatures = X.shape[1]
    
    gw = np.full(nfeatures, 0.0)
    gb = np.full(1, 0.0)
    return gw, gb

# fit an SVM using gradient descent
# X: matrix of feature values
# y: labels (-1 or 1)
# lam: penalty parameter lambda
# n_iter: numer of iterations
# eta: learning rate
def fit_svm(X, y, lam, n_iter=100, eta=.4):
    nexamples, nfeatures = X.shape
    
    w, b = _initialize_parameters(nfeatures)
    
    for k in range(n_iter):
        f = _get_signed_distances(X, w, b)
        
        # print information and 
        # update the learning rate
        if k % 10 == 0:
            obj = _get_objective(f, y, w, lam)
            eta = eta / 2.0
            print("it: %d, obj %.2f" % (k, obj))
        
        gw, gb = _get_gradients(f, X, y, w, b, lam)
        w = w - eta * gw
        b = b - eta * gb
    return w, b

In [9]:
w,b = fit_svm(train_X, train_y, 1.0, n_iter=100)

NameError: name 'train_X' is not defined